## <a id="summary">SUMMARY</a>


- [1. Gathering](#gathering)
- [2. Assessing](#assessing)
    - [2.1 tweet_infos](#assessing_tweet_infos)
    - [2.2 image_predictions](#assessing_image_predictions)
    - [2.3 missing_info](#assessing_missing_info)
    - [Issues Notes](#issues_notes)
- [3. Cleaning](#clean)


In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os

## <a id="gathering">1. Gathering</a>
[Go to Summary](#summary)

In [2]:
# Downloading image-predictions.tsv programatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open("image-predictions.tsv", "wb") as file:
    file.write(response.content)

In [3]:
# Loading twitter-archive-enhanced.csv and image-predictions.tsv which has been downloader programatically in the above cell
tweet_infos = pd.read_csv("twitter-archive-enhanced.csv", sep=",")
image_predictions = pd.read_csv("image-predictions.tsv", sep="\t")

In [4]:
consumer_key = 'liLZ73eupzfhJLPBcxzGDBVtY'
consumer_secret = 'sj8QRUCVF4zBFB5bYef0e9NHZhCJ13Uj9rUF2Gs0mCcZVwJjAz'
access_token = '1052682995868598274-iE1y3grIvRXNRJm1GXkhZHCdOFVXEK'
access_secret = 'vv1yINXwWXLAoc875PyWr4REtVqJNqtO5HOwd6KwXv4kM'

In [5]:
# Acessando TWEEPY API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
# Loading missing_info.json if it exists, 
# if not, get its information from tweepy, save it as missing_info.json, then load it
if 'missing_info.json' in os.listdir():
    missing_info = pd.read_json("missing_info.json")
else:
    missing_info_list = []
    tweet_ids = tweet_infos.tweet_id.unique()
    for tweet_id in tweet_ids:
        try:
            status = api.get_status(tweet_id)
            retweet_count = status.retweet_count
            favorite_count = status.favorite_count

            missing_info_list.append({"tweet_id": str(tweet_id), 
                                 "retweet_count": str(retweet_count), 
                                 "favorite_count": str(favorite_count)})

        except tweepy.error.TweepError:
            continue

    with open("missing_info.json", "w") as file:
        file.write(json.dumps(missing_info_list))

    missing_info = pd.read_json("missing_info.json")

## <a id=assessing>2. Assessing</a>
[Go to Summary](#summary)


### <a id="assessing_tweet_infos">2.1 DataFrame tweet_infos </a>
[Go to Summary](#summary)

In [7]:
tweet_infos.sample(30)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
208,852311364735569921,NaN,NaN,2017-04-13 00:03:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Wiggles. She would like you to spot he...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/852311364...,13,10,Wiggles,None,None,None,None
2103,670676092097810432,NaN,NaN,2015-11-28 18:50:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bloo. He's a Westminster Cîroc. Doesn'...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670676092...,8,10,Bloo,None,None,None,None
653,792050063153438720,NaN,NaN,2016-10-28 17:07:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Remus. He's a mop that came to life. C...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/792050063...,11,10,Remus,None,None,None,None
511,811985624773361665,NaN,NaN,2016-12-22 17:23:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Ted. He accidentally opened the f...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/811985624...,11,10,Ted,None,None,None,None
1950,673688752737402881,NaN,NaN,2015-12-07 02:21:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Larry. He doesn't know how to shoe. 9/10 ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673688752...,9,10,Larry,None,None,None,None
392,826115272272650244,NaN,NaN,2017-01-30 17:10:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ike. He's demonstrating the pupmost re...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/826115272...,13,10,Ike,None,None,None,None
1434,697270446429966336,NaN,NaN,2016-02-10 04:06:43 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bentley. He got stuck on his 3rd homew...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697270446...,10,10,Bentley,None,None,None,None
463,817423860136083457,NaN,NaN,2017-01-06 17:33:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ken. His cheeks are magic. 13/10 (IG: ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/817423860...,13,10,Ken,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
84,876484053909872640,NaN,NaN,2017-06-18 16:57:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Benedict. He wants to thank you for th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/876484053...,13,10,Benedict,None,None,None,None


In [8]:
tweet_infos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
name_value_counts = tweet_infos.name.value_counts()
strange_names = name_value_counts[name_value_counts.index.str.islower()].index.tolist()
strange_names

['a',
 'the',
 'an',
 'very',
 'quite',
 'one',
 'just',
 'getting',
 'not',
 'actually',
 'mad',
 'space',
 'his',
 'unacceptable',
 'light',
 'incredibly',
 'old',
 'infuriating',
 'all',
 'this',
 'officially',
 'by',
 'my',
 'life',
 'such']

In [10]:
tweet_infos.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [11]:
tweet_infos[tweet_infos["rating_numerator"] > 20][["tweet_id", "expanded_urls", "text", "rating_numerator"]]

,tweet_id,expanded_urls,text,rating_numerator
188,855862651834028034,NaN,@dhmontgomery We also gave snoop dogg a 420/10...,420
189,855860136149123072,NaN,@s8n You tried very hard to portray this good ...,666
290,838150277551247360,NaN,@markhoppus 182/10,182
313,835246439529840640,NaN,@jonnysun @Lin_Manuel ok jomny I know you're e...,960
340,832215909146226688,https://twitter.com/dog_rates/status/786709082...,"RT @dog_rates: This is Logan, the Chow who liv...",75
433,820690176645140481,https://twitter.com/dog_rates/status/820690176...,The floofs have been released I repeat the flo...,84
516,810984652412424192,"https://www.gofundme.com/sams-smile,https://tw...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,24
695,786709082849828864,https://twitter.com/dog_rates/status/786709082...,"This is Logan, the Chow who lived. He solemnly...",75
763,778027034220126208,https://twitter.com/dog_rates/status/778027034...,This is Sophie. She's a Jubilant Bush Pupper. ...,27
902,758467244762497024,https://twitter.com/dog_rates/status/758467244...,Why does this never happen at my front door......,165


In [12]:
tweet_infos[tweet_infos["rating_denominator"] != 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None


In [13]:
tweet_infos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [14]:
tweet_infos.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [15]:
tweet_infos.timestamp.value_counts()

2016-03-31 23:22:53 +0000    1
2016-11-30 18:16:08 +0000    1
2016-12-27 17:36:16 +0000    1
2016-02-23 02:12:47 +0000    1
2015-11-24 02:38:07 +0000    1
                            ..
2016-03-04 16:06:36 +0000    1
2015-11-19 21:22:56 +0000    1
2016-12-13 18:01:07 +0000    1
2016-03-22 02:03:52 +0000    1
2016-01-19 01:18:43 +0000    1
Name: timestamp, Length: 2356, dtype: int64

In [16]:
tweet_infos.timestamp.min(), tweet_infos.timestamp.max()

('2015-11-15 22:32:08 +0000', '2017-08-01 16:23:56 +0000')

In [17]:
tweet_infos[tweet_infos.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### <a id="assessing_image_predictions">2.2 DataFrame image_predictions</a>
[Go to Summary](#summary)

In [18]:
image_predictions.sample(30)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1231,745789745784041472,https://pbs.twimg.com/media/ClmT0KHWkAAXbhy.jpg,1,Pekinese,0.984267,True,Shih-Tzu,0.008942,True,cocker_spaniel,0.001928,True
1230,745712589599014916,https://pbs.twimg.com/media/CllNnkWWMAEDIAR.jpg,1,seat_belt,0.379055,False,chow,0.062754,True,minibus,0.052423,False
169,668989615043424256,https://pbs.twimg.com/media/CUi6geuUYAIvE9n.jpg,1,pug,0.917326,True,waffle_iron,0.014918,False,Chihuahua,0.013524,True
772,689557536375177216,https://pbs.twimg.com/media/CZHM60BWIAA4AY4.jpg,1,Eskimo_dog,0.169482,True,Siberian_husky,0.161655,True,dingo,0.154414,False
454,674764817387900928,https://pbs.twimg.com/media/CV0_BSuWIAIvE9k.jpg,2,Samoyed,0.634695,True,Arctic_fox,0.309853,False,kuvasz,0.019641,True
797,691090071332753408,https://pbs.twimg.com/media/CZc-u7IXEAQHV1N.jpg,1,barrow,0.241637,False,tub,0.238450,False,bathtub,0.167285,False
1023,710283270106132480,https://pbs.twimg.com/media/Cdtu3WRUkAAsRVx.jpg,2,Shih-Tzu,0.932401,True,Lhasa,0.030806,True,Tibetan_terrier,0.008974,True
1686,814638523311648768,https://pbs.twimg.com/media/C04taUjWIAA6Mo4.jpg,2,golden_retriever,0.650814,True,kuvasz,0.053281,True,cocker_spaniel,0.035440,True
1234,746369468511756288,https://pbs.twimg.com/media/ClujESVXEAA4uH8.jpg,1,German_shepherd,0.622957,True,malinois,0.338884,True,wallaby,0.024161,False
1947,862722525377298433,https://pbs.twimg.com/media/C_kBjuUUIAArs2-.jpg,1,basset,0.393330,True,beagle,0.242034,True,boxer,0.077692,True


In [19]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [20]:
total = image_predictions["p1_conf"] +  image_predictions["p2_conf"] + image_predictions["p3_conf"]
total[total > 1]

106    1.0
dtype: float64

In [21]:
total[106]

1.000000010287017

In [22]:
image_predictions.p1.value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
hammer                  1
prayer_rug              1
Japanese_spaniel        1
military_uniform        1
bighorn                 1
Name: p1, Length: 378, dtype: int64

In [23]:
image_predictions.p2.value_counts()

Labrador_retriever    104
golden_retriever       92
Cardigan               73
Chihuahua              44
Pomeranian             42
                     ... 
breakwater              1
cannon                  1
lifeboat                1
shower_curtain          1
hair_spray              1
Name: p2, Length: 405, dtype: int64

In [24]:
image_predictions.p3.value_counts()

Labrador_retriever    79
Chihuahua             58
golden_retriever      48
Eskimo_dog            38
kelpie                35
                      ..
greenhouse             1
zebra                  1
cheetah                1
rhinoceros_beetle      1
spatula                1
Name: p3, Length: 408, dtype: int64

In [25]:
image_predictions[image_predictions["tweet_id"].duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


### <a id="assessing_missing_info">2.3 DataFrame missing_info</a>
[Go to Summary](#summary)

In [26]:
missing_info.sample(30)

,tweet_id,retweet_count,favorite_count
653,789960241177853952,4218,4218
81,876838120628539392,3059,3059
781,772581559778025472,1718,1718
70,878776093423087616,3744,3744
663,788765914992902144,10572,10572
1434,666073100786774016,147,147
511,808501579447930880,2670,2670
439,818307523543449600,8250,8250
1211,670411370698022912,851,851
374,826240494070030336,2640,2640


In [27]:
missing_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 3 columns):
tweet_id          1447 non-null int64
retweet_count     1447 non-null int64
favorite_count    1447 non-null int64
dtypes: int64(3)
memory usage: 34.0 KB


In [28]:
missing_info[missing_info["tweet_id"].duplicated()]

,tweet_id,retweet_count,favorite_count


## <a id="issues_notes">Issues</a>
[Go to Summary](#summary)

### Quality
**All DataFrames**
- Column `tweet_id` is int, should be object

**Dataframe** `tweet_infos`
- Columns `name`, `doggo`, `floofer`, `pupper`, `puppo` have None string for Null values
- Lowercase dog names are not real dog names. The strange names are in the strange_names variable
- Some urls are null. We can't do anything about it.
- Some numerators are greater than 20 and some denominatores are different than 10 (Dica: verificar os textos desses casos em q numerador é maior que 20 e denominador diferente de 10 e verificar se podemos aplicar uma regra para corrigir vários numeradores e denominadores de uma só vez. Essa mudança será parecida com a correção de nomes que fizemos)
- Column `expanded_urls` sometimes have more than the tweet URL (Dica: verificar se existe um padrão nesses registros com mais de uma URL, exemplo: url_ruim,url_do_post Se a url do tweet estiver sempre depois da vírgula, um .str.split(",", expand=True) pode ajudar.
- Columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` , `retweeted_status_timestamp ` most values are null. (Não acredito que há muito o que fazer, uma abordagem seria utilizar a API do twitter para verificar se conseguimos usá-la para adquirir esses campos de lá.)
- All values from column `source` have HTML tags (Dica: uma alternativa é procurar na internet algum regex que pega o texto entre tags HTML. Caso não encontrem, segunda alternativa é usar o split (primeiro split da sourge em >, e pegar o que vem depois desse caractere, após, uma abordagem pode fatiar essa string, tirando os últimos três caractéres ([:-3]) ou um replace("</a>", "") )
- Column `timestamp` is object, should be datetime


<font color='darkpink'>**TO DO**</font>

**Dataframe** `image_predictions`
- Columns `p1`, `p2`, `p3` don't have a string format pattern (Dica: replaces e métodos string como .title() resolvem).
- Columns `p1`, `p2`, `p3` and its variations names don't are so descriptive.

### Tidiness
**Dataframe** `tweet_infos`
- Columns `doggo`, `floofer`, `pupper`, `puppo` are values of the same variable: dog stages (Dica: verificar se essas informações estão sendo tiradas do texto do tweet, caso estejam, utilizar regex na coluna "text" com a função .str.extract. O regex seria '(doggo|floofer|pupper|puppo)'. Caso queiram uma alternativa: usar .apply no dataframe e, para cada iteração, verificar o que tem nas colunas doogo, floofer, puppper, puppo. Aplicar lógica: se todos forem nulos, retorna nulo, caso apenas um não seja nulo, reporta esse único que não é nulo. Se houver casos em que há mais de um diferente de nulo, temos que verificar o texto desse tweet, afinal, um dog não pode estar em 2 fases da vida ao mesmo tempo.)

**Dataframe** `missing_info`
- Shoul be part of `tweet_infos` dataframe (Dica: usar o merge sobre a coluna "tweet_id", o join deve priorizar os elementos que estão na tabela tweet_infos, portanto, um left join (caso o df tweet_infos esteja à esquerda, caso ele esteja a direita, deve-se usar right join) é mais interessante.

In [29]:
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


## <font color='#b031ff'><a id="clean">3. CLEAN</a></font>
[Go to Summary](#summary)

In [30]:
# Make dataframe copies
ti_clean = tweet_infos.copy()
ip_clean = image_predictions.copy()
mi_clean = missing_info.copy()

**Issue**: Column tweet_id is int, should be object

**Plan**: Parse to object

In [31]:
ti_clean["tweet_id"] = ti_clean["tweet_id"].astype("str")
ip_clean["tweet_id"] = ip_clean["tweet_id"].astype("str")
mi_clean["tweet_id"] = mi_clean["tweet_id"].astype("str")

**Test**

In [32]:
mi_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 3 columns):
tweet_id          1447 non-null object
retweet_count     1447 non-null int64
favorite_count    1447 non-null int64
dtypes: int64(2), object(1)
memory usage: 34.0+ KB


**Issue**: Columns name, doggo, floofer, pupper, puppo have None string for Null values

**Plan**: Change None by np.nan

In [33]:
cols_oi = ["name", "doggo", "floofer", "pupper", "puppo"]
ti_clean[cols_oi] = ti_clean[cols_oi].replace("None", np.nan)

**Test**

In [34]:
ti_clean.tail(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2346,666058600524156928,NaN,NaN,2015-11-16 01:01:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is the Rand Paul of retrievers folks! He'...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666058600...,8,10,the,NaN,NaN,NaN,NaN
2347,666057090499244032,NaN,NaN,2015-11-16 00:55:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",My oh my. This is a rare blond Canadian terrie...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666057090...,9,10,a,NaN,NaN,NaN,NaN
2348,666055525042405380,NaN,NaN,2015-11-16 00:49:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a Siberian heavily armored polar bear ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666055525...,10,10,a,NaN,NaN,NaN,NaN
2349,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853...,2,10,an,NaN,NaN,NaN,NaN
2350,666050758794694657,NaN,NaN,2015-11-16 00:30:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a truly beautiful English Wilson Staff...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666050758...,10,10,a,NaN,NaN,NaN,NaN
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,NaN,NaN,NaN,NaN,NaN
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,NaN,NaN,NaN,NaN
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,NaN,NaN,NaN,NaN
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,NaN,NaN,NaN,NaN
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,NaN,NaN,NaN,NaN,NaN


**Issue**: Lowercase dog names are not real dog names. The strange names are in the strange_names variable

**Plan**: Check if we can use some regex to correct these names, if so, change them, if not: 
- check if another rule can be applied V
- correct each name one by one X
- make these names null V

In [35]:
# Filtrando dataframe. Quero apenas os registros que tem algum nome na lista strange_names
filtered_df = ti_clean[ti_clean["name"].isin(strange_names)]

# Pego o conteúdo do tweet desse dataframe filtrado
# Com essa variável, conseguimos
texts = filtered_df.text.tolist()

# Pego os indices que estão no dataframe filtrado, ou seja os indices de registros cujo nome do dog está estranho
indices_strange_names = filtered_df.index

In [36]:
# Definindo função que nos ajudará a pegar nomes que estão após a pelavra named e o ponto (.)
def pick_name(row):
    index = row["index"]
    text = row["text"].lower()
    current_name = row["name"]
    
    if index not in indices_strange_names and not pd.isna(current_name):
        return current_name
    
    if "named" in text:
        name = text.split("named")[1].split(".")[0].strip().title()
        return name
    else:
        return np.nan

# Uso o apply sobre ti_clean.reset_index com drop = False, pois queremos que o índice vire coluna
# O índice é importante na função pick name
# Além disso, aplicamos a regra aos registros cuja coluna name era nula
ti_clean["name"] = ti_clean.reset_index(drop=False).apply(pick_name, axis=1)

**Test**

In [37]:
ti_clean.name.value_counts()[:50]

Charlie    12
Oliver     11
Lucy       11
Cooper     11
Lola       10
Penny      10
Tucker     10
Winston     9
Bo          9
Sadie       8
Daisy       7
Toby        7
Buddy       7
Bailey      7
Bella       6
Koda        6
Dave        6
Jack        6
Rusty       6
Milo        6
Oscar       6
Leo         6
Stanley     6
Jax         6
Scout       6
Bentley     5
Louis       5
Sunny       5
Alfie       5
Finn        5
Phil        5
Chester     5
Sammy       5
Larry       5
George      5
Gus         5
Oakley      5
Shadow      4
Ruby        4
Moose       4
Archie      4
Jeffrey     4
Boomer      4
Jerry       4
Derek       4
Carl        4
Loki        4
Luna        4
Riley       4
Bruce       4
Name: name, dtype: int64

In [38]:
ti_clean.name.value_counts()[-50:]

Fiji            1
Pherb           1
Anthony         1
Maks            1
Snickers        1
Edmund          1
Jordy           1
Leonard         1
Darla           1
DonDon          1
Augie           1
Kayla           1
Obie            1
Ralphus         1
Meera           1
Jed             1
Halo            1
Arlo            1
Julio           1
Yukon           1
Alexanderson    1
Divine          1
Wishes          1
Tayzie          1
Stark           1
Sailer          1
Rizzo           1
Gunner          1
Tilly           1
Dutch           1
Vince           1
Kobe            1
Jockson         1
Nigel           1
Jarvis          1
Carper          1
Grizzie         1
Bubba           1
Coopson         1
Jazzy           1
Pawnd           1
Sonny           1
Hero            1
Diogi           1
Fynn            1
Dante           1
Gustaf          1
Durg            1
Link            1
Laela           1
Name: name, dtype: int64

**Issue**: Some numerators are greater than 20 and some denominatores are different than 10

**Plan**: Verificar os textos desses casos em q numerador é maior que 20 e denominador diferente de 10 e verificar se podemos aplicar uma regra para corrigir vários numeradores e denominadores de uma só vez. Essa mudança será parecida com a correção de nomes que fizemos

In [39]:
# Filtrando os registros que tem numerador > 20 OU denominador != 10
filt_df = ti_clean[(ti_clean["rating_numerator"] > 20) | (ti_clean["rating_denominator"] != 10)]

# A partir do filt_df, printar os textos e seus respectivos indices para avaliar quais indices devemos corrigir,
# e qual devemos setar como nulos
texts = filt_df.text.tolist()
indices = filt_df.index.tolist()

for i, text in zip(indices, texts):
    print(i, text, "\n")

188 @dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research 

189 @s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10 

290 @markhoppus 182/10 

313 @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho 

340 RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu… 

342 @docmisterio account started on 11/15/15 

433 The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd 

516 Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx 

695 This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS 

763 This is Sophie. She's a 

In [40]:
# As listas abaixo foram preenchidas com a análise feita acima
indices_to_remove_n_d = [290, 342, 516]
indices_to_correct = [313, 340, 695, 763, 784, 1068, 1165, 1202, 1662, 1712, 2335]

# Seto o numerador de nominador dos índices que não posso corrigir como nulo
ti_clean.loc[indices_to_remove_n_d, ["rating_numerator", "rating_denominator"]] = np.nan

In [41]:
# Itero sobre todo o dataframe
# Para aqueles índices que não precisamos corrigir, não faço nada: retorno o próprio numerador e o próprio denominador.
# Para aqueles índices que estão na lista indices_to_correct, aplico a regra definida com splits
def correct_ratings_nums(row, num):
    index = row["index"]
    if index not in indices_to_correct:
        return row[num]
    
    else: 
        raw_text = row["text"]
        wo_https_text = raw_text.split(" https:")[0]
        splitted_text = wo_https_text.split("/")
        
        if num == "rating_numerator":
            return splitted_text[-2].split(" ")[-1]
        elif num == "rating_denominator":
            return splitted_text[-1].split(" ")[0]
    
    
ti_clean["rating_numerator"] = ti_clean.reset_index(drop=False)\
            .apply(lambda row: correct_ratings_nums(row, "rating_numerator"), axis=1).astype("float64")
ti_clean["rating_denominator"] = ti_clean.reset_index(drop=False)\
            .apply(lambda row: correct_ratings_nums(row, "rating_denominator"), axis=1).astype("float64")

**Test**

In [ ]:
filt_df = ti_clean[(ti_clean["rating_denominator"] != 10)]

ti_clean.info()

**Issue**: Column `expanded_urls` sometimes have more than the tweet URL

**Plan**: Verificar se existe um padrão nesses registros com mais de uma URL, exemplo: url_ruim,url_do_post Se a url do tweet estiver sempre depois da vírgula, um .str.split(",", expand=True) pode ajudar.

In [ ]:
ti_clean.info()

In [ ]:
# Filtro registros cuja coluna expanded_urls tem vírgula
filt_df = ti_clean[ti_clean.fillna("")["expanded_urls"].str.contains(",")]

# Pego o último elemento de um split na vírgula da coluna expanded_urls
splitted_url = filt_df["expanded_urls"].str.split(",").str[-1]

# Filtro da lista acima, os elementos que não contém twitter.com e pego seus índices
indices_of_strange_urls = splitted_url[~splitted_url.str.contains("twitter.com")].index

# No dataframe original, printo a coluna expanded_urls dos índices pegos acima como lista para verificar como são essas
ti_clean.loc[indices_of_strange_urls, "expanded_urls"].tolist()

In [ ]:
ti_clean["expanded_urls"] = ti_clean["expanded_urls"].str.split(",").str[-1]

**Test**

In [ ]:
ti_clean[ti_clean.fillna("oi")["expanded_urls"].str.contains(",")]

**Issue** All values from column source have HTML tags

**Plan** Uma alternativa é procurar na internet algum regex que pega o texto entre tags HTML. Caso não encontrem, segunda alternativa é usar o split (primeiro split da sourge em >, e pegar o que vem depois desse caractere, após, uma abordagem pode fatiar essa string, tirando os últimos três caractéres ([:-3]) ou um replace("", "")

In [ ]:
def get_rid_of_html_tags(row):
    source = row["source"]
    
    return source.split("</a>")[0].split(">")[-1]

ti_clean["source"] = ti_clean.apply(get_rid_of_html_tags, axis=1)

**Test**

In [ ]:
ti_clean.source.value_counts()

**Issue**: Column `timestamp` is object, should be datetime

**Plan**: Change it to datetime

In [ ]:
ti_clean["timestamp"].head(5)

In [ ]:
ti_clean["timestamp"] = pd.to_datetime(ti_clean["timestamp"], format="%Y-%m-%d %H:%M:%S %z")

**Test**

In [ ]:
ti_clean.dtypes["timestamp"]